In [89]:
import random
import glob
import json
import numpy as np

## Fetch the data

In [90]:
def get_card_as_one_hot(card):
    s = ['clubs','hearts','spades','diamonds'].index(card['suit'])
    v = ([ str(i) for i in range(2,11) ] + ['J','K','Q','A']).index(card['rank'])
    return v,s

def get_cards_as_one_hot(cards):
    one_hot = np.zeros((13,4))
    for c in cards:
        one_hot[get_card_as_one_hot(c)] = 1
    return one_hot

def get_all_cards_for_hand_for_player(hand, player_index):
    return np.array([
        get_cards_as_one_hot(hand['hole_cards'][player_index]), 
        get_cards_as_one_hot(hand['community_cards'][:3]),
        get_cards_as_one_hot(hand['community_cards'][3:4]),
        get_cards_as_one_hot(hand['community_cards'][4:5])
    ])

def rotate(a, n):
    return a[n:] + a[:n]

def normalize(bets, active_player, blind):
    result = rotate(bets,active_player)
    return list(map(lambda x: x / blind, result))

def get_bets(hand, active_player):
    bets=[]
    number_of_players = len(hand['bets'][0][0])
    previous_bet = 0
    bet_index = 0
    for r in hand['bets']:
        bets.append([])
        for b in r:
            if b['on_turn'] == active_player:
                if bet_index > 1: 
                    current_bet = b['bets'][active_player]- previous_bet 
                    normalized_totals = normalize(b['bets'], active_player, hand['blind'])
                    normalized_current_bet = (current_bet)/hand['blind']
                    normalized_totals[0] -= normalized_current_bet
                    
                    bets[-1].append({ 
                        'bet': normalized_current_bet,
                        'totals': normalized_totals,
                        'type': b['type']
                    })
            previous_bet = b['bets'][active_player]
            bet_index += 1
    return bets


def zero_last_layers(layers, round_index):
    for i in range(round_index + 1, len(layers)):
        layers[i] = np.zeros(layers[i].shape)
    return layers

def one_hot(i, length):
    one_hot = np.zeros((length,))
    one_hot[i] = 1
    return one_hot

def encode_action_type(action):
    return one_hot({
        'check': 0,
        'fold': 0,
        'call': 1,
        'raise': 2,
        'allin': 3
    }[action], 4)

def extend_bet_vector(bet_matrix):
    return np.c_[bet_matrix, np.zeros((3, 6 - bet_matrix.shape[1]))]

def convert_hand(hand):
    inputs = []
    outputs = []
    for round_index, round_bets in enumerate(get_bets(hand, hand['game_winner'])):
        for this_bet in round_bets:
            nof_players = len(this_bet['totals'])
            normalized_dealer = (hand['dealer'] - hand['game_winner'] + nof_players) % nof_players
            cards = get_all_cards_for_hand_for_player(hand, hand['game_winner'])
            initial_stacks = normalize(hand['initial_stacks'], hand['game_winner'], hand['blind'])
            remaining_stack = (np.array(initial_stacks) - np.array(this_bet['totals']))
            dealer_onehot = one_hot(normalized_dealer, nof_players)
            
            bet_vector = np.array([remaining_stack, this_bet['totals'],dealer_onehot])
            
            my_bet = bet_vector[:, 0]
                
            bet_vector = extend_bet_vector(bet_vector[:,1:]).reshape(3,6,1)
            
            inputs.append([
                zero_last_layers(cards, round_index), # cards
                my_bet,
                bet_vector
            ])
            outputs.append(encode_action_type(this_bet['type']))
    return inputs, outputs


In [91]:
file = glob.glob("/data/poker/rawdata/*.json")[3]
f = open(file, 'r')
hands = json.loads(f.read())
test_hand = hands[3]

convert_hand(test_hand)[0][0]

[array([[[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.]],
 
        [[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
  

In [92]:
inputs = []
outputs = []

for file in glob.glob("/data/poker/rawdata/*.json"):
    f = open(file, 'r')
    hands_in_game = json.loads(f.read())

    for hand in hands_in_game: 
        inp, outp = convert_hand(hand)
        inputs.extend(inp)
        outputs.extend(outp)

inputs = np.array(inputs)
outputs = np.array(outputs)

In [93]:
inputs.shape

(450222, 3)

In [94]:
outputs.shape

(450222, 4)

In [95]:
import bcolz

ifile = bcolz.carray(inputs, rootdir='/data/poker/predict_action_input_2.dat', mode='w')
ifile.flush()
ofile = bcolz.carray(outputs, rootdir='/data/poker/predict_action_output_2.dat', mode='w')
ofile.flush()